<a href="https://colab.research.google.com/github/irene880104/datasharing/blob/master/Tailored_sourcing_Irene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

| Product | Demand | Common | Specific | Unit cost | Holding cost rate |
|---------|--------|--------|----------|-----------|-------------------|
|    1    |  1000  |  100   |   10     |    50     |        0.2        |
|    2    |   300  |  100   |   20     |    60     |        0.2        |
|    3    |   100  |  100   |   25     |    30     |        0.2        |
|    4    |   50   |  100   |   25     |    30     |        0.2        |


## import data

In [2]:
import pandas as pd
from math import sqrt

data = {
    'Product': [1, 2, 3, 4],
    'Demand': [1000, 300, 100, 50],
    'Common': [100, 100, 100, 100],
    'Specific': [10, 20, 25, 25],
    'Unit cost': [50, 60, 30, 30],
    'Holding cost rate': [0.2, 0.2, 0.2, 0.2]
}

df = pd.DataFrame(data)

df


,Product,Demand,Common,Specific,Unit cost,Holding cost rate
0,1,1000,100,10,50,0.2
1,2,300,100,20,60,0.2
2,3,100,100,25,30,0.2
3,4,50,100,25,30,0.2


𝐶𝑜𝑠𝑡 𝑄 = 𝐻𝑜𝑙𝑑𝑖𝑛𝑔 𝑐𝑜𝑠𝑡 𝑄 + 𝑂𝑟𝑑𝑒𝑟𝑖𝑛𝑔 𝐶𝑜𝑠𝑡 𝑄 + 𝑀𝑎𝑡𝑒𝑟𝑖𝑎𝑙 𝑐𝑜𝑠𝑡 = 𝐻*𝑄/2 + 𝑆*𝐷/𝑄 + 𝐷𝐶

Optimal Q = sqrt(2DS/hc)

# 1 Products are sourced independently


In [3]:
def optimal_q(D, S, h, c ):
    optimal_q = (2 * D * S / (h*c)) ** 0.5
    return round(optimal_q,2)

In [4]:
q1 = df.copy()
q1['optimal_order_size'] = optimal_q(q1['Demand'], q1['Common']+ q1['Specific'], q1['Holding cost rate'], q1['Unit cost'])
q1['order_frequency'] = q1['Demand']/ q1['optimal_order_size']
q1['annual_cost'] = q1['Holding cost rate']*q1['Unit cost'] *q1['optimal_order_size']/2 +( q1['Common']+ q1['Specific'])*q1['Demand']/q1['optimal_order_size']
q1

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,optimal_order_size,order_frequency,annual_cost
0,1,1000,100,10,50,0.2,148.32,6.742179,1483.239698
1,2,300,100,20,60,0.2,77.46,3.872967,929.516003
2,3,100,100,25,30,0.2,64.55,1.549187,387.298335
3,4,50,100,25,30,0.2,45.64,1.095530,273.861280


In [5]:
round(q1['annual_cost'].sum(),2)

3073.92

# 2 All four products are sourced with the same frequency;



In [13]:
S = 100 + df['Specific'].sum()
q2 = df.copy()
q2['order_frequency'] = (((df['Demand'] * df['Holding cost rate'] * df['Unit cost']).sum() / (2 * S)) ** 0.5)

q2['optimal_order_size'] = q2['Demand']/q2['order_frequency']
q2['annual_order_cost'] =  q2['order_frequency']  *S
q2['annual_holding_cost'] = q2['Holding cost rate']*q2['Unit cost'] *q2['optimal_order_size']/2
q2

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,order_frequency,optimal_order_size,annual_order_cost,annual_holding_cost
0,1,1000,100,10,50,0.2,6.346478,157.567719,1142.365966,787.838597
1,2,300,100,20,60,0.2,6.346478,47.270316,1142.365966,283.621895
2,3,100,100,25,30,0.2,6.346478,15.756772,1142.365966,47.270316
3,4,50,100,25,30,0.2,6.346478,7.878386,1142.365966,23.635158


In [15]:
round(q2['annual_holding_cost'].sum()+q2['annual_order_cost'].max(),2)

2284.73

# 3 Order frequencies are determined according to the tailored aggregation strategy.

In [109]:
q3 = df.copy()

In [110]:
q3

,Product,Demand,Common,Specific,Unit cost,Holding cost rate
0,1,1000,100,10,50,0.2
1,2,300,100,20,60,0.2
2,3,100,100,25,30,0.2
3,4,50,100,25,30,0.2


## Step 1: most frequent order product -- product 1

In [111]:
q1['order_frequency'].max()

6.74217907227616

## Step 2: other product frequency

In [112]:
q3_without = q3.iloc[1:, :].copy()
q3_without

,Product,Demand,Common,Specific,Unit cost,Holding cost rate
1,2,300,100,20,60,0.2
2,3,100,100,25,30,0.2
3,4,50,100,25,30,0.2


## Step 3: frequency calculate

In [113]:
max = q1['order_frequency'].max()
q3_without['optimal_order_size'] = optimal_q(q3_without['Demand'], q3_without['Specific'], q3_without['Holding cost rate'], q3_without['Unit cost'])
q3_without['order_frequency'] = (q3_without['Demand']/ q3_without['optimal_order_size'])
q3_without['m'] = round(max/q3_without['order_frequency'],0)

q3_without

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,optimal_order_size,order_frequency,m
1,2,300,100,20,60,0.2,31.62,9.487666,1.0
2,3,100,100,25,30,0.2,28.87,3.463803,2.0
3,4,50,100,25,30,0.2,20.41,2.449780,3.0


## Step 4: recalculate

In [115]:
# Assuming q3 and q3_without are DataFrames
q3_new = q3_without.copy()
row_to_add = q3.iloc[0, :]
q3_new.loc[len(q3_new)+1] = row_to_add
q3_new.iloc[3,7] = max
q3_new.iloc[3,6] = q1['optimal_order_size'].iloc[0]
q3_new.iloc[3,8] = 1
q3_new

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,optimal_order_size,order_frequency,m
1,2.0,300.0,100.0,20.0,60.0,0.2,31.62,9.487666,1.0
2,3.0,100.0,100.0,25.0,30.0,0.2,28.87,3.463803,2.0
3,4.0,50.0,100.0,25.0,30.0,0.2,20.41,2.449780,3.0
4,1.0,1000.0,100.0,10.0,50.0,0.2,148.32,6.742179,1.0


In [117]:
def n(h, C, D, m, S, s):
    n = ((h * C * D * m).sum() / (2 * (S + (s/m).sum())))**(1/2)
    return n

h = q3_new['Holding cost rate']
C = q3_new['Unit cost']
D = q3_new['Demand']
m = q3_new['m']
S = 100
s = q3_new['Specific']

n_pro_1 = n(h, C, D, m, S, s)
print(n_pro_1)

7.21416654195255


In [118]:
q3_new['new_frequency'] = n_pro_1/q3_new['m']
#q3_new.iloc[3,9] = n_pro_1
q3_new

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,optimal_order_size,order_frequency,m,new_frequency
1,2.0,300.0,100.0,20.0,60.0,0.2,31.62,9.487666,1.0,7.214167
2,3.0,100.0,100.0,25.0,30.0,0.2,28.87,3.463803,2.0,3.607083
3,4.0,50.0,100.0,25.0,30.0,0.2,20.41,2.449780,3.0,2.404722
4,1.0,1000.0,100.0,10.0,50.0,0.2,148.32,6.742179,1.0,7.214167


In [119]:
q3_new['new_optimal_size'] = q3_new['Demand']/q3_new['new_frequency']
q3_new

,Product,Demand,Common,Specific,Unit cost,Holding cost rate,optimal_order_size,order_frequency,m,new_frequency,new_optimal_size
1,2.0,300.0,100.0,20.0,60.0,0.2,31.62,9.487666,1.0,7.214167,41.584845
2,3.0,100.0,100.0,25.0,30.0,0.2,28.87,3.463803,2.0,3.607083,27.723230
3,4.0,50.0,100.0,25.0,30.0,0.2,20.41,2.449780,3.0,2.404722,20.792423
4,1.0,1000.0,100.0,10.0,50.0,0.2,148.32,6.742179,1.0,7.214167,138.616151


In [145]:
hold_cost = (q3_new['new_optimal_size']*q3_new['Holding cost rate']*q3_new['Unit cost']/2).sum()

In [146]:
order_cost = (q3_new['Demand'] *q3_new['Specific']/q3_new['new_optimal_size']).sum()

In [147]:
hold_cost + order_cost +q3_new['new_frequency'].max()*100

2176.2735734890193